In [1]:
import pandas as pd

In [ ]:
"""guardian_archive.csv -- Файл с собранными новостями с 01.01.2017 по 01.11.2023."""
df = pd.read_csv('guardian_archive.csv')
df.columns

In [ ]:
"""ВНИМАНИЕ! Это вспомогательные функции, которые может быть понадобятся потом.
Используем их для того, если вдруг нужно будет из csv файла создать таблицу и положить в бд.
1. Настраивает соединение с бд.
2. Создает таблицу в базе данных.
Таким образом, можно будет использовать в других шагах проекта"""
import psycopg2
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
def create_connection_to_db(path):#1
    connection = None
    try:
        connection = psycopg2.connect(dbname=path, user='user', 
                        password='password', host='localhost')
        print("Connection to PostgreSQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_table_and_insert_data_from_csv(connection, csv_file_path, table_name):#2

    df = pd.read_csv(csv_file_path)
    df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)#использовать, если есть ненужные столбцы 
    
    # Устанавливаем соединение с базой данных
    cursor = connection.cursor()
    
    #Создаем запрос на создание таблицы с первичным ключом news_id и колонками из датафрейма с типом данных TEXT(по необходимости)
    create_table_query = f'CREATE TABLE IF NOT EXISTS {table_name} (news_id SERIAL PRIMARY KEY, {" ,".join([f"{column} TEXT" for column in df.columns])})'
    
    # Создаем запрос для внесения данных в таблицу
    insert_query = f'INSERT INTO {table_name} ({", ".join(df.columns)}) VALUES ({", ".join(["%s"] * len(df.columns))})'
    
    # Выполняем SQL-запрос для создания таблицы
    cursor.execute(create_table_query)
    
    # Вставляем данные
    for _, row in df.iterrows():
        cursor.execute(insert_query, row)
    
    # комитимся
    connection.commit()
    
    # Закрываем курсор
    cursor.close()

# Создаем подключение к PostgreSQL
connection = create_connection_to_db('postgres')#dbname=path

# Создаем таблицу и вставляем данные из CSV-файла
create_table_and_insert_data_from_csv(connection, 'guardian_archive.csv', 'news_guardian')

# Закрываем соединение с базой данных
connection.close()

In [ ]:
#обновление таблицы новостей и записи в логи(пока только текстовый файл)
import requests
from datetime import datetime, timedelta, date
import logging
import pandas as pd
import psycopg2
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Настройка логирования
logging.basicConfig(filename='update_database.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - update_database - %(message)s')


# Функция для создания соединения с базой данных PostgreSQL. Создал отдельно еще одну для удобства
def create_connection(database_name, user, password, host, port):
    try:
        connection = psycopg2.connect(
            database=database_name,
            user=user,
            password=password,
            host=host,
            port=port
        )
        connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        return connection
    except Error as e:
        raise Exception(f"Ошибка при подключении к базе данных: {str(e)}")

def update_database(start_date, end_date):
    success = False  # Флаг успешного обновления. Изначально считаем, что обновление неуспешно
    try:
        # Чтение API-ключа из файла
        with open('api_key.txt', 'r') as api_key_file:
            api_key = api_key_file.read().strip()

        # Чтение ключевых слов из файла
        with open('tags.txt', 'r') as file:
            tags = [line.strip() for line in file]

        result_data = []

        while start_date <= end_date:
            # Объединяем теги в одну строку через '|'. В документации означет OR
            tags_query = '|'.join(tags)

            params = {
                'api-key': api_key,
                'q': tags_query,
                'from-date': start_date,
                'to-date': end_date,
                'page-size': 100,
                'show-fields': ['body', 'bodyText']
            }

            url = 'https://content.guardianapis.com/search'

            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = response.json()
                for article in data['response']['results']:
                    # Определяем список тегов, которые были найдены в тексте статьи или в заголовке
                    found_tags = [tag for tag in tags if (tag.lower() in article['webTitle'].lower()) or (tag.lower() in article['fields']['bodyText'].lower())]
                    if found_tags:
                        result_data.append({
                            'type': article['type'],
                            'sectionId': article['sectionId'],
                            'sectionName': article['sectionName'],
                            'webPublicationDate': article['webPublicationDate'],
                            'webTitle': article['webTitle'],
                            'text': article['fields']['bodyText'],
                            'webUrl': article['webUrl'],
                            'apiUrl': article['apiUrl'],
                            'searchedTags': '/'.join(found_tags)  # Используем только найденные теги
                        })

                #Соединяем теги через '/' и переводим теги слитно в вид CamelCase
                df = pd.DataFrame(result_data)
                df['searchedTags'] = df['searchedTags'].apply(lambda tags: '/'.
                                                              join([''.join(word.title() for word in tag.split())
                                                                    if ' ' in tag else tag for tag in tags.split('/')]))
                tags = df['searchedTags'].str.split('/').sum()
                unique_tags = list(set(tags))

                #Создаем колонки для тегов. Пока ставим признак, что тег отсутствует
                for tag in unique_tags:
                    df[tag] = 0

                # Ставим 1 к колонке с признаком, если он есть в searchedTags
                for index, row in df.iterrows():
                    for tag in row['searchedTags'].split('/'):
                        if tag in unique_tags:
                            df.at[index, tag] = 1

                # Преобразование даты 
                df['webPublicationDate'] = pd.to_datetime(df['webPublicationDate']).dt.strftime('%Y-%m-%d')
                # Подключаемся к базе данных PostgreSQL
                connection = create_connection('db_name', 'user', 'password', 'localhost', '5432')
                cursor = connection.cursor()

                # Вставляем данные в таблицу. По итогу найденные новости и созданные признаки тегов отправляются в БД
                # На ошибку при вставке не нарываемся, так как признакам поставлен параметр DEFAULT 0
                for _, row in df.iterrows():
                    placeholders = ', '.join(['%s'] * len(unique_tags))
                    columns = ', '.join(unique_tags)
                    query = f"INSERT INTO news_guardian (type, sectionId, sectionName, webPublicationDate, webTitle, text, webUrl, apiUrl, searchedTags, {columns}) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, {placeholders})"
    
                    cursor.execute(query, (row['type'], row['sectionId'], row['sectionName'], row['webPublicationDate'], row['webTitle'],
                            row['text'], row['webUrl'], row['apiUrl'], row['searchedTags'], *[row[tag] for tag in unique_tags]))

                connection.commit()
                cursor.close()

                start_date = (date.fromisoformat(start_date) + timedelta(days=1)).isoformat()
            else:
                error_message = f"Ошибка при запросе к API: {response.status_code}"
                logging.error(error_message)
                print(error_message)
                raise Exception(error_message)# Эти ошибки возникают при сбое в работе с API

        success = True  # Обновление успешно завершено. В консоль успех не выводим
        logging.info("Обновление базы данных завершено успешно!")

    except Exception as e: #Остальные ошибки
        error_message = f"Произошла ошибка: {str(e)}"
        logging.error(error_message)
        print(error_message)

    if not success:
        logging.warning("Обновление базы данных завершилось с ошибкой!")

if __name__ == "__main__":
    start_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    end_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

    update_database(start_date, end_date)